## HybridGibbs
One feature we get from the new sampler design is more flexibility in our Gibbs sampling.
Now we can for example run with sampler instances, which allows us to set parameters and also use all samplers in Gibbs sampling.

We call this new sampler HybridGibbs to distinguish it from classical Gibbs sampling.

### Old Gibbs sampler
First let us showcase the old Gibbs sampler:

In [ ]:
import cuqi
import numpy as np
from cuqi.distribution import Gamma, Gaussian, GMRF, JointDistribution, LMRF
from cuqi.experimental.mcmc import NUTS, HybridGibbs, Conjugate, LinearRTO, ConjugateApprox, UGLA
from cuqi.testproblem import Deconvolution1D

# Forward problem
A, y_data, info = Deconvolution1D(dim=128, phantom='sinc', noise_std=0.001).get_components()

# Bayesian Inverse Problem
s = Gamma(1, 1e-4)
x = GMRF(np.zeros(A.domain_dim), 50)
y = Gaussian(A@x, lambda s: 1/s)

# Posterior
target = JointDistribution(y, x, s)(y=y_data)


In [ ]:
# Old-style Gibbs. Only supports "Static" (non-adaptive) samplers
sampling_strategy = {
    "x" : cuqi.sampler.LinearRTO,
    "s" : cuqi.sampler.Conjugate
}

sampler = cuqi.sampler.Gibbs(target, sampling_strategy)

# Sample
samples = sampler.sample(200, 50)

# Plot results
samples["x"].plot_ci()

The new HybridGibbs sampler is a generalization of the old Gibbs sampler which supported object-oriented sampler classes.

This means we can use more elaborate Gibbs sampling schemes like NUTS-within-Gibbs or similar.

First let us explore how the interface has changed by replicating the previous case:

In [ ]:
# New-style Hybrid Gibbs. Supports object-oriented that maintain state through iterations

sampling_strategy = {
    "x" : LinearRTO(maxit=25), # <--- Notice instance of class with set parameters
    "s" : Conjugate()          # <--- Notice instance of class with default parameters
}

sampler = HybridGibbs(target, sampling_strategy)

# Sample using new-style Gibbs
sampler.warmup(50)
sampler.sample(200)
samples = sampler.get_samples()

# Plot results
samples["x"].plot_ci()


Now let us try the NUTS-within-Gibbs sampler.

In [ ]:
sampling_strategy = {
    "x" : NUTS(max_depth=10),
    "s" : Conjugate()
}

# Here we do 10 internal steps with NUTS for each Gibbs step
num_sampling_steps = {
    "x" : 10,
    "s" : 1
}

sampler = HybridGibbs(target, sampling_strategy, num_sampling_steps)

sampler.warmup(50)
sampler.sample(200)
samples = sampler.get_samples()

samples["x"].plot_ci()

The approximate conjugate case (from the webpage) also works

In [ ]:
# Try with ConjugateApprox (LMRF prior)

# Forward problem
A, y_data, info = Deconvolution1D(dim=128, phantom='square', noise_std=0.001).get_components()

# Bayesian Inverse Problem
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)
x = LMRF(0, lambda d: 1/d, geometry=A.domain_geometry)
y = Gaussian(A@x, lambda s: 1/s)

# Posterior
target = JointDistribution(y, x, s, d)(y=y_data)

# Sampling strategy
sampling_strategy = {
    "x" : UGLA(),
    "s" : Conjugate(),
    "d" : ConjugateApprox()
}

# Gibbs sampler
sampler = HybridGibbs(target, sampling_strategy)

# Run sampler
sampler.warmup(50)
sampler.sample(200)
samples = sampler.get_samples()

# Plot
samples["x"].plot_ci()

## Ensuring conjugacy safety and flexibility
One important feature of the new Conjugate samplers is that they check the parameter relations to ensure an actual conjugate distribution is defined in the model. This was previously not checked in the older sampler module.

This will help users not accidentally using the conjugate sampler for cases that are not actually conjugate.

In [ ]:
# Forward problem
A, y_data, info = Deconvolution1D(dim=128, phantom='sinc', noise_std=0.001).get_components()

# Baysian Inverse Problem
s = Gamma(1, 1e-4)
x = GMRF(np.zeros(A.domain_dim), 50)
y = Gaussian(A@x, lambda s: s) # Accidentally using wrong conjugate relation (should have been 1/s)

# Posterior
target = JointDistribution(y, x, s)(y=y_data)

# Sampling strategy
sampling_strategy = {
    "x" : LinearRTO(maxit=25),
    "s" : Conjugate()
}

# Gibbs sampler (this should print an error)
try:
    sampler = HybridGibbs(target, sampling_strategy)
except ValueError as e:
    print("Sampler error thrown: ", e)
